In [1]:
import pandas as pd
import os

print(pd.__version__)

2.0.3


In [2]:
small_set_data = pd.read_parquet("data/Zzzs_train.parquet")

In [3]:
print(small_set_data)

             series_id    step                 timestamp     anglez    enmo  \
0         08db4255286f       0  2018-11-05T10:00:00-0400 -30.845301  0.0447   
1         08db4255286f       1  2018-11-05T10:00:05-0400 -34.181801  0.0443   
2         08db4255286f       2  2018-11-05T10:00:10-0400 -33.877102  0.0483   
3         08db4255286f       3  2018-11-05T10:00:15-0400 -34.282101  0.0680   
4         08db4255286f       4  2018-11-05T10:00:20-0400 -34.385799  0.0768   
...                ...     ...                       ...        ...     ...   
13165555  d5e47b94477e  395815  2017-12-02T15:44:35-0500 -10.064100  0.1552   
13165556  d5e47b94477e  395816  2017-12-02T15:44:40-0500 -10.006700  0.1139   
13165557  d5e47b94477e  395817  2017-12-02T15:44:45-0500  -3.328600  0.1245   
13165558  d5e47b94477e  395818  2017-12-02T15:44:50-0500  -5.903100  0.1363   
13165559  d5e47b94477e  395819  2017-12-02T15:44:55-0500 -11.691100  0.0995   

          awake  
0             1  
1             1

In [4]:
data_series = {}
for series in set(small_set_data["series_id"]):
    data_series[series] = small_set_data[small_set_data["series_id"] == series]

In [5]:
print(data_series['18b61dd5aae8'].iloc[0:10])

            series_id  step                 timestamp     anglez    enmo  \
1885140  18b61dd5aae8     0  2017-12-22T16:30:00-0500 -14.944400  0.0091   
1885141  18b61dd5aae8     1  2017-12-22T16:30:05-0500 -15.538300  0.0204   
1885142  18b61dd5aae8     2  2017-12-22T16:30:10-0500 -15.362400  0.0048   
1885143  18b61dd5aae8     3  2017-12-22T16:30:15-0500 -15.332100  0.0008   
1885144  18b61dd5aae8     4  2017-12-22T16:30:20-0500 -14.292500  0.0065   
1885145  18b61dd5aae8     5  2017-12-22T16:30:25-0500 -16.090900  0.0087   
1885146  18b61dd5aae8     6  2017-12-22T16:30:30-0500 -16.971300  0.0206   
1885147  18b61dd5aae8     7  2017-12-22T16:30:35-0500 -15.477400  0.0124   
1885148  18b61dd5aae8     8  2017-12-22T16:30:40-0500 -18.168501  0.0660   
1885149  18b61dd5aae8     9  2017-12-22T16:30:45-0500 -12.208900  0.1289   

         awake  
1885140      1  
1885141      1  
1885142      1  
1885143      1  
1885144      1  
1885145      1  
1885146      1  
1885147      1  
1885148   

In [2]:
window_size = 50

In [6]:
def create_sliding_windows_W_buffer(data,window_size,buffer,feature):
    column_names = ["local step " + str(i) for i in range(window_size)]
    column_names = ["series_id", "global first step num", "global first timestamp"]+column_names
    column_names = column_names + ["awake","wakeup","fallasleep","asleep"]
    windowed_data = []
    dummy_counter = 10
    for i in range(len(data) - window_size + 1):
        current_row = list(data.iloc[i,0:3]) + list(data[feature][i:i+window_size])
        #print(i, sum(data["enmo"][i:i+window_size]))
        if sum(data["awake"][i:i+window_size]) >= window_size - buffer: #Therefore during a time window, if the majority of the timesteps are
            #awake then the window is defined as awake; it does not matter if the window started with awake as 0, or 1.
            state = [1,0,0,0]
        elif sum(data["awake"][i:i+window_size]) <= buffer:   
            state = [0,0,0,1]
        elif data["awake"].iloc[i] == 1:   
            state = [0,0,1,0]
        elif data["awake"].iloc[i] == 0:   
            state = [0,1,0,0] 
        current_row += state
        #print(len(current_row))
        #print(current_row)
        #print(type(windowed_data))
        windowed_data.append(current_row)
        if (i*100) // (len(data) - window_size + 1) % 10 == 0 and (i  * 100 )// (len(data) - window_size + 1) != 0 and (i*100) // (len(data) - window_size + 1) >= dummy_counter:
            print(f"series {data.iloc[i,0]}  completed: {(i*100) // (len(data) - window_size + 1)} %")
            dummy_counter += 10
    windowed_data = pd.DataFrame(windowed_data, columns=column_names) #windowed_data.append(pd.Series(current_row, index=windowed_data.columns), ignore_index=True)
        
    return windowed_data
    
        
        
        
    

In [9]:
#This function was outdated on 10/22/2023
def create_sliding_windows_anglez(data,window_size):
    column_names = ["local step " + str(i) for i in range(window_size)]
    column_names = ["series_id", "global first step num", "global first timestamp"]+column_names
    column_names = column_names + ["awake","wakeup","fallasleep","asleep"]
    windowed_data = []
    dummy_counter = 10
    for i in range(len(data) - window_size + 1):
        current_row = list(data.iloc[i,0:3]) + list(data["anglez"][i:i+window_size])
        #print(i, sum(data["enmo"][i:i+window_size]))
        if sum(data["awake"][i:i+window_size]) == window_size:
            state = [1,0,0,0]
        elif sum(data["awake"][i:i+window_size]) == 0:   
            state = [0,0,0,1]
        elif data["awake"].iloc[i] == 1:   
            state = [0,0,1,0]
        elif data["awake"].iloc[i] == 0:   
            state = [0,1,0,0] 
        current_row += state
        #print(len(current_row))
        #print(current_row)
        #print(type(windowed_data))
        windowed_data.append(current_row)
        if (i*100) // (len(data) - window_size + 1) % 10 == 0 and (i  * 100 )// (len(data) - window_size + 1) != 0 and (i*100) // (len(data) - window_size + 1) >= dummy_counter:
            print(f"series {data.iloc[i,0]}  completed: {(i*100) // (len(data) - window_size + 1)} %")
            dummy_counter += 10
    windowed_data = pd.DataFrame(windowed_data, columns=column_names) #windowed_data.append(pd.Series(current_row, index=windowed_data.columns), ignore_index=True)
        
    return windowed_data

In [23]:
print(partitions['18b61dd5aae8'][2])

            series_id   step                 timestamp     anglez    enmo  \
1897237  18b61dd5aae8  12097  2017-12-23T09:18:05-0500  63.511398     0.0   
1897238  18b61dd5aae8  12098  2017-12-23T09:18:10-0500    55.4753  0.0057   
1897239  18b61dd5aae8  12099  2017-12-23T09:18:15-0500  54.178398  0.0084   
1897240  18b61dd5aae8  12100  2017-12-23T09:18:20-0500  62.499298     0.0   
1897241  18b61dd5aae8  12101  2017-12-23T09:18:25-0500  62.499298  0.0001   
...               ...    ...                       ...        ...     ...   
1906808  18b61dd5aae8  21668  2017-12-23T22:35:40-0500   -13.8579  0.0139   
1906809  18b61dd5aae8  21669  2017-12-23T22:35:45-0500   -13.6616  0.0139   
1906810  18b61dd5aae8  21670  2017-12-23T22:35:50-0500   -13.3994  0.0154   
1906811  18b61dd5aae8  21671  2017-12-23T22:35:55-0500   -13.5491  0.0207   
1906812  18b61dd5aae8  21672  2017-12-23T22:36:00-0500   -13.6627  0.0162   

        awake  
1897237     1  
1897238     1  
1897239     1  
1897240    

In [28]:
import os
# Assuming 'partitions' is your dictionary
for key, df_list in partitions.items():
    for index, df in enumerate(df_list):
        # Get the value of the first row of the "awake" column
        first_row_value = df["awake"].iloc[0]

        # Generate a unique file name based on key, index, and first row value
        file_name = f"{key}_index{index}_awake{first_row_value}.csv"
        
        # Define the path where the CSV file will be saved
        if first_row_value == 0:
            save_path = os.path.join("data/asleep", file_name)
        if first_row_value == 1:
            save_path = os.path.join("data/awake", file_name)

        # Save the DataFrame to the CSV file
        df.to_csv(save_path, index=False)

In [7]:
import os

In [90]:



window_sizes = [50, 100, 200, 400, 800]

for window_size in window_sizes:
    windowed_dataset = pd.DataFrame()
    for series in set(small_set_data["series_id"]):
    
    
        
        windowed_series = create_sliding_windows(data_series[series] ,window_size = window_size)
        windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
    file_name = f"{window_size}_small_set_enmo_windowed.csv"
    save_path = os.path.join("data/windowed_sequences", file_name)
    windowed_dataset.to_csv(save_path, index=False)

series 8e32047cbc1f  completed: 10 %
series 8e32047cbc1f  completed: 20 %
series 8e32047cbc1f  completed: 30 %
series 8e32047cbc1f  completed: 40 %
series 8e32047cbc1f  completed: 50 %
series 8e32047cbc1f  completed: 60 %
series 8e32047cbc1f  completed: 70 %
series 8e32047cbc1f  completed: 80 %
series 8e32047cbc1f  completed: 90 %
series c6788e579967  completed: 10 %
series c6788e579967  completed: 20 %
series c6788e579967  completed: 30 %
series c6788e579967  completed: 40 %
series c6788e579967  completed: 50 %
series c6788e579967  completed: 60 %
series c6788e579967  completed: 70 %
series c6788e579967  completed: 80 %
series c6788e579967  completed: 90 %
series 7822ee8fe3ec  completed: 10 %
series 7822ee8fe3ec  completed: 20 %
series 7822ee8fe3ec  completed: 30 %
series 7822ee8fe3ec  completed: 40 %
series 7822ee8fe3ec  completed: 50 %
series 7822ee8fe3ec  completed: 60 %
series 7822ee8fe3ec  completed: 70 %
series 7822ee8fe3ec  completed: 80 %
series 7822ee8fe3ec  completed: 90 %
s

MemoryError: Unable to allocate 35.7 GiB for an array with shape (400, 11972931) and data type float64

In [14]:
#window_sizes = [50, 100, 200, 400, 800]
window_sizes = [100, 200]
for window_size in window_sizes:
    windowed_dataset = pd.DataFrame()
    for series in set(small_set_data["series_id"]):
    
    
        
        windowed_series = create_sliding_windows_anglez(data_series[series] ,window_size = window_size)
        windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
    file_name = f"{window_size}_small_set_anglez_windowed.csv"
    save_path = os.path.join("data/windowed_sequences", file_name)
    windowed_dataset.to_csv(save_path, index=False)

series c535634d7dcd  completed: 10 %
series c535634d7dcd  completed: 20 %
series c535634d7dcd  completed: 30 %
series c535634d7dcd  completed: 40 %
series c535634d7dcd  completed: 50 %
series c535634d7dcd  completed: 60 %
series c535634d7dcd  completed: 70 %
series c535634d7dcd  completed: 80 %
series c535634d7dcd  completed: 90 %
series 8e32047cbc1f  completed: 10 %
series 8e32047cbc1f  completed: 20 %
series 8e32047cbc1f  completed: 30 %
series 8e32047cbc1f  completed: 40 %
series 8e32047cbc1f  completed: 50 %
series 8e32047cbc1f  completed: 60 %
series 8e32047cbc1f  completed: 70 %
series 8e32047cbc1f  completed: 80 %
series 8e32047cbc1f  completed: 90 %
series a9e5f5314bcb  completed: 10 %
series a9e5f5314bcb  completed: 20 %
series a9e5f5314bcb  completed: 30 %
series a9e5f5314bcb  completed: 40 %
series a9e5f5314bcb  completed: 50 %
series a9e5f5314bcb  completed: 60 %
series a9e5f5314bcb  completed: 70 %
series a9e5f5314bcb  completed: 80 %
series a9e5f5314bcb  completed: 90 %
s

MemoryError: Unable to allocate 11.8 MiB for an array with shape (200, 483) and data type <U32

In [15]:
    save_path = os.path.join("data/windowed_sequences", file_name)
    windowed_dataset.to_csv(save_path, index=False)

In [17]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_anglez(data_series["5f94bb3e1bed"] ,window_size = 50)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed.csv"
save_path = os.path.join("data/windowed_sequences", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


Generating one-series windowed dataset W buffers, with different window sizes, and different buffer sizes for experiment.

In [15]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 100,buffer=20,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W100_B20.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [16]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 100,buffer=20,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W100_B20.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [17]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 100,buffer=30,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W100_B30.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 100,buffer=30,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W100_B30.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [18]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=20,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W200_B20.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=20,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W200_B20.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [19]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=0,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W200_B0.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=0,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W200_B0.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [20]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=40,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W200_B40.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=40,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W200_B40.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [24]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=60,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W200_B60.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 200,buffer=60,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W200_B60.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [21]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=40,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W400_B40.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=40,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W400_B40.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [22]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=0,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W400_B0.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=0,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W400_B0.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [23]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=80,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W400_B80.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=80,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W400_B80.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [25]:

windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=120,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_anglez_windowed_W400_B120.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series["5f94bb3e1bed"] ,window_size = 400,buffer=120,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"one_series_small_set_enmo_windowed_W400_B120.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %
series 5f94bb3e1bed  completed: 10 %
series 5f94bb3e1bed  completed: 20 %
series 5f94bb3e1bed  completed: 30 %
series 5f94bb3e1bed  completed: 40 %
series 5f94bb3e1bed  completed: 50 %
series 5f94bb3e1bed  completed: 60 %
series 5f94bb3e1bed  completed: 70 %
series 5f94bb3e1bed  completed: 80 %
series 5f94bb3e1bed  completed: 90 %


In [8]:
series= "18b61dd5aae8"
windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series[series] ,window_size = 400,buffer=40,feature = 'anglez')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"{series}_small_set_anglez_windowed_W400_B40.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)


windowed_dataset = pd.DataFrame()
windowed_series = create_sliding_windows_W_buffer(data_series[series] ,window_size = 400,buffer=40,feature = 'enmo')
#create_sliding_windows_W_buffer(data,window_size,buffer=0,feature)
windowed_dataset = pd.concat([windowed_dataset, windowed_series], ignore_index=True)
file_name = f"{series}_small_set_enmo_windowed_W400_B40.csv"
save_path = os.path.join("data/windowed_sequences/W_buffer", file_name)
windowed_dataset.to_csv(save_path, index=False)

series 18b61dd5aae8  completed: 10 %
series 18b61dd5aae8  completed: 20 %
series 18b61dd5aae8  completed: 30 %
series 18b61dd5aae8  completed: 40 %
series 18b61dd5aae8  completed: 50 %
series 18b61dd5aae8  completed: 60 %
series 18b61dd5aae8  completed: 70 %
series 18b61dd5aae8  completed: 80 %
series 18b61dd5aae8  completed: 90 %
series 18b61dd5aae8  completed: 10 %
series 18b61dd5aae8  completed: 20 %
series 18b61dd5aae8  completed: 30 %
series 18b61dd5aae8  completed: 40 %
series 18b61dd5aae8  completed: 50 %
series 18b61dd5aae8  completed: 60 %
series 18b61dd5aae8  completed: 70 %
series 18b61dd5aae8  completed: 80 %
series 18b61dd5aae8  completed: 90 %
